<a href="https://colab.research.google.com/github/SelenaNahra/DL/blob/main/HW5P4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

# Special tokens and hyperparameters
SOS_token = 0  # Start Of Sequence Token
EOS_token = 1  # End Of Sequence Token
hidden_size = 512
num_layers = 5
nhead = 4
learning_rate = 0.0001
epochs = 100

# French to English dataset
french_to_english = [
    ("J'ai froid", "I am cold"),
    ("Tu es fatigué", "You are tired"),
    ("Il a faim", "He is hungry"),
    ("Elle est heureuse", "She is happy"),
    ("Nous sommes amis", "We are friends"),
    ("Ils sont étudiants", "They are students"),
    ("Le chat dort", "The cat is sleeping"),
    ("Le soleil brille", "The sun is shining"),
    ("Nous aimons la musique", "We love music"),
    ("Elle parle français couramment", "She speaks French fluently"),
    ("Il aime lire des livres", "He enjoys reading books"),
    ("Ils jouent au football chaque week-end", "They play soccer every weekend"),
    ("Le film commence à 19 heures", "The movie starts at 7 PM"),
    ("Elle porte une robe rouge", "She wears a red dress"),
    ("Nous cuisinons le dîner ensemble", "We cook dinner together"),
    ("Il conduit une voiture bleue", "He drives a blue car"),
    ("Ils visitent souvent des musées", "They visit museums often"),
    ("Le restaurant sert une délicieuse cuisine", "The restaurant serves delicious food"),
    ("Elle étudie les mathématiques à l'université", "She studies mathematics at university"),
    ("Nous regardons des films le vendredi", "We watch movies on Fridays"),
    ("Il écoute de la musique en faisant du jogging", "He listens to music while jogging"),
    ("Ils voyagent autour du monde", "They travel around the world"),
    ("Le livre est sur la table", "The book is on the table"),
    ("Elle danse avec grâce", "She dances gracefully"),
    ("Nous célébrons les anniversaires avec un gâteau", "We celebrate birthdays with cake"),
    ("Il travaille dur tous les jours", "He works hard every day"),
    ("Ils parlent différentes langues", "They speak different languages"),
    ("Les fleurs fleurissent au printemps", "The flowers bloom in spring"),
    ("Elle écrit de la poésie pendant son temps libre", "She writes poetry in her free time"),
    ("Nous apprenons quelque chose de nouveau chaque jour", "We learn something new every day"),
    ("Le chien aboie bruyamment", "The dog barks loudly"),
    ("Il chante magnifiquement", "He sings beautifully"),
    ("Ils nagent dans la piscine", "They swim in the pool"),
    ("Les oiseaux gazouillent le matin", "The birds chirp in the morning"),
    ("Elle enseigne l'anglais à l'école", "She teaches English at school"),
    ("Nous prenons le petit déjeuner ensemble", "We eat breakfast together"),
    ("Il peint des paysages", "He paints landscapes"),
    ("Ils rient de la blague", "They laugh at the joke"),
    ("L'horloge tic-tac bruyamment", "The clock ticks loudly"),
    ("Elle court dans le parc", "She runs in the park"),
    ("Nous voyageons en train", "We travel by train"),
    ("Il écrit une lettre", "He writes a letter"),
    ("Ils lisent des livres à la bibliothèque", "They read books at the library"),
    ("Le bébé pleure", "The baby cries"),
    ("Elle étudie dur pour les examens", "She studies hard for exams"),
    ("Nous plantons des fleurs dans le jardin", "We plant flowers in the garden"),
    ("Il répare la voiture", "He fixes the car"),
    ("Ils boivent du café le matin", "They drink coffee in the morning"),
    ("Le soleil se couche le soir", "The sun sets in the evening"),
    ("Elle danse à la fête", "She dances at the party"),
    ("Nous jouons de la musique au concert", "We play music at the concert"),
    ("Il cuisine le dîner pour sa famille", "He cooks dinner for his family"),
    ("Ils étudient la grammaire française", "They study French grammar"),
    ("La pluie tombe doucement", "The rain falls gently"),
    ("Elle chante une chanson", "She sings a song"),
    ("Nous regardons un film ensemble", "We watch a movie together"),
    ("Il dort profondément", "He sleeps deeply"),
    ("Ils voyagent à Paris","They travel to Paris"),
    ("Les enfants jouent dans le parc","The children play in the park"),
    ("Elle se promène le long de la plage","She walks along the beach"),
    ("Nous parlons au téléphone","We talk on the phone"),
    ("Il attend le bus","He waits for the bus"),
    ("Ils visitent la tour Eiffel","They visit the Eiffel Tower"),
    ("Les étoiles scintillent la nuit","The stars twinkle at night"),
    ("Elle rêve de voler", "She dreams of flying"),
    ("Nous travaillons au bureau", "We work in the office"),
    ("Il étudie l’histoire", "He studies history"),
    ("Ils écoutent la radio", "They listen to the radio"),
    ("Le vent souffle doucement", "The wind blows gently"),
    ("Elle nage dans l’océan", "She swims in the ocean"),
    ("Nous dansons au mariage", "We dance at the wedding"),
    ("Il gravit la montagne", "He climbs the mountain"),
    ("Ils font de la randonnée dans la forêt", "They hike in the forest"),
    ("Le chat miaule bruyamment", "The cat meows loudly"),
    ("Elle peint un tableau", "She paints a picture"),
    ("Nous construisons un château de sable", "We build a sandcastle"),
    ("Il chante dans le chœur", "He sings in the choir")
    ]


# Building separate vocabularies for French and English sentences
def build_vocab(sentences):
    vocab = {"SOS": SOS_token, "EOS": EOS_token}
    for sentence in sentences:
        for word in sentence.split():
            if word not in vocab:
                vocab[word] = len(vocab)
    return vocab

french_vocab = build_vocab(fr for fr, _ in french_to_english)
english_vocab = build_vocab(en for _, en in french_to_english)

# Custom Dataset class for handling French-English sentence pairs
class FrenchEnglishDataset(Dataset):
    def __init__(self, data, src_vocab, trg_vocab):
        self.data = data
        self.src_vocab = src_vocab
        self.trg_vocab = trg_vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        src_sentence, trg_sentence = self.data[idx]
        src_indices = [self.src_vocab[word] for word in src_sentence.split()] + [EOS_token]
        trg_indices = [self.trg_vocab[word] for word in trg_sentence.split()] + [EOS_token]
        return torch.tensor(src_indices, dtype=torch.long), torch.tensor(trg_indices, dtype=torch.long)

# DataLoader with custom collate function to handle padding
def collate_fn(batch):
    src_batch, tgt_batch = zip(*batch)
    src_batch_padded = pad_sequence([torch.tensor(seq) for seq in src_batch], batch_first=True, padding_value=EOS_token)
    tgt_batch_padded = pad_sequence([torch.tensor(seq) for seq in tgt_batch], batch_first=True, padding_value=EOS_token)
    return src_batch_padded, tgt_batch_padded

dataset = FrenchEnglishDataset(french_to_english, french_vocab, english_vocab)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

# Transformer model definition
class TranslatorTransformer(nn.Module):
    def __init__(self, input_vocab_size, output_vocab_size, hidden_size, num_layers, nhead):
        super(TranslatorTransformer, self).__init__()
        self.embedding_src = nn.Embedding(input_vocab_size, hidden_size)
        self.embedding_tgt = nn.Embedding(output_vocab_size, hidden_size)
        self.transformer = nn.Transformer(d_model=hidden_size, nhead=nhead, num_encoder_layers=num_layers,
                                          num_decoder_layers=num_layers, batch_first=True)
        self.fc_out = nn.Linear(hidden_size, output_vocab_size)

    def forward(self, src, tgt):
        src = self.embedding_src(src)
        tgt = self.embedding_tgt(tgt)
        output = self.transformer(src, tgt)
        output = self.fc_out(output)
        return output

# Model, loss, and optimizer setup
model = TranslatorTransformer(len(french_vocab), len(english_vocab), hidden_size, num_layers, nhead)
criterion = nn.CrossEntropyLoss(ignore_index=EOS_token)  # Ignore padding for loss calculation
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop with conditional loss printing
def train_model(model, dataloader, optimizer, criterion, epochs, print_every=10):
    model.train()
    for epoch in range(1, epochs + 1):
        total_loss = 0
        for src, tgt in dataloader:
            optimizer.zero_grad()
            tgt_input = tgt[:, :-1]
            tgt_output = tgt[:, 1:]
            outputs = model(src, tgt_input)
            outputs = outputs.reshape(-1, outputs.shape[-1])
            tgt_output = tgt_output.reshape(-1)
            loss = criterion(outputs, tgt_output)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        if epoch % print_every == 0:
            print(f'Epoch {epoch}: Loss {total_loss / len(dataloader)}')

# Run the training
train_model(model, dataloader, optimizer, criterion, epochs)

def evaluate_and_show_examples(model, dataloader, criterion, src_vocab, trg_vocab, n_examples=5):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_sentences = 0

    idx_to_word_src = {idx: word for word, idx in src_vocab.items()}
    idx_to_word_trg = {idx: word for word, idx in trg_vocab.items()}

    with torch.no_grad():
        for i, (src, tgt) in enumerate(dataloader):
            output = model(src, tgt[:, :-1])  # Feed the target as input to the decoder, excluding the EOS token at the end
            output = output.reshape(-1, output.shape[-1])
            tgt_output = tgt[:, 1:].reshape(-1)  # Shifted target outputs
            loss = criterion(output, tgt_output)
            total_loss += loss.item()

            # Calculate accuracy
            _, predicted_indices = torch.max(output, dim=1)
            correct_predictions += (predicted_indices == tgt_output).sum().item()
            total_sentences += tgt_output.size(0)

            if i < n_examples:
                # Print examples
                src_words = [idx_to_word_src[idx.item()] for idx in src[0] if idx.item() in idx_to_word_src]
                trg_words = [idx_to_word_trg[idx.item()] for idx in tgt[0] if idx.item() in idx_to_word_trg]
                pred_words = [idx_to_word_trg[idx.item()] for idx in predicted_indices.view(tgt.size(0), -1)[0] if idx.item() in idx_to_word_trg]

                print(f'Input: {" ".join(src_words)}')
                print(f'Target: {" ".join(trg_words)}')
                print(f'Predicted: {" ".join(pred_words)}')
                print("")

        average_loss = total_loss / len(dataloader)
        accuracy = correct_predictions / total_sentences
        print(f'Evaluation Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}')

<ipython-input-14-2a0b90e8af57>:129: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_batch_padded = pad_sequence([torch.tensor(seq) for seq in src_batch], batch_first=True, padding_value=EOS_token)
<ipython-input-14-2a0b90e8af57>:130: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_batch_padded = pad_sequence([torch.tensor(seq) for seq in tgt_batch], batch_first=True, padding_value=EOS_token)


Epoch 10: Loss 1.8159321302022688
Epoch 20: Loss 0.08066602003497955
Epoch 30: Loss 0.02485238925482218
Epoch 40: Loss 0.014392848699711837
Epoch 50: Loss 0.009572335865157537
Epoch 60: Loss 0.0068356769087796025
Epoch 70: Loss 0.005062362710491587
Epoch 80: Loss 0.0038868156261742115
Epoch 90: Loss 0.003006064121086055
Epoch 100: Loss 0.002429117615788411


In [16]:
evaluate_and_show_examples(model, dataloader, criterion, french_vocab, english_vocab, n_examples=5)

<ipython-input-14-2a0b90e8af57>:129: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_batch_padded = pad_sequence([torch.tensor(seq) for seq in src_batch], batch_first=True, padding_value=EOS_token)
<ipython-input-14-2a0b90e8af57>:130: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_batch_padded = pad_sequence([torch.tensor(seq) for seq in tgt_batch], batch_first=True, padding_value=EOS_token)


Input: Ils visitent souvent des musées EOS EOS
Target: They visit museums often EOS
Predicted: visit museums often often

Input: Il peint des paysages EOS
Target: He paints landscapes EOS
Predicted: paints landscapes landscapes

Input: Nous parlons au téléphone EOS EOS
Target: We talk on the phone EOS
Predicted: talk on the phone on

Input: Le soleil brille EOS
Target: The sun is shining EOS
Predicted: sun is shining is

Input: Le soleil se couche le soir EOS
Target: The sun sets in the evening EOS
Predicted: sun sets in the evening in

Evaluation Loss: 0.0005, Accuracy: 0.7036
